In [1]:
import os
from bs4 import BeautifulSoup
import time
import datetime
import pandas as pd
import requests
import re

from datetime import datetime, timedelta
import pause, sys

# 크롤링
def crawling():
    
    # 섹션 별 아이디 설정
    Section_ID = {"Political":"100", "Economic":"101", "Society":"102", "Culture":"103", "International":"104", "Science":"105"}

    # 시간 별 크롤링 시간 측정
    total_crawling_time = 0
    now = datetime.now()
    print("Start Crawling :", now.date(), now.hour)
    
    # 파일 저장 경로 설정
    date = "".join(str(now.date()).split('-'))
    directory = "C:\\Users\\cjy89\\NLP\\Project_news_crawling\\Naver\\" + str(date)
    if not os.path.isdir(directory):
        os.makedirs(directory)
    os.chdir(directory)

    # 섹션 별로 크롤링
    for section in Section_ID:

        # 섹션 별 크롤링 시간 측정 start
        start = time.time()
        page = 0
        if section + ".csv" in os.listdir(directory):
            News_DataFrame = pd.read_csv(section+".csv", encoding="utf-8-sig", engine="python")
        else:
            News_DataFrame = pd.DataFrame()

        # 서버에서 뉴스 데이터 가져오기
        flag = True
        while(flag):
            page += 1

            # 제목형으로 정치 섹션 데이터 긁어오기
            url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&listType=title&sid1=" + Section_ID["Political"] + "&date=" + date + "&page=" + str(page)
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
            resp = requests.get(url, headers=headers)
            soup = BeautifulSoup(resp.text, "html.parser")

            # news_box 한 요소당 5개의 뉴스가 포함됨
            news_box = soup.find_all(class_ = "type02")
            for box in range(len(news_box)):
                news = [i.get_text() for i in news_box[box].find_all(class_="nclicks(fls.list)")]
                urls = [i['href'] for i in news_box[box].find_all(class_="nclicks(fls.list)")]
                press = [i.get_text() for i in news_box[box].find_all(class_="writing")]
                upload = [ str(now.date()) + ' ' + str(now.hour - 1) + ':' + str(60 + (now.minute - int(i.get_text()[:-2]))) 
                          for i in news_box[box].find_all(class_="date is_new") if i.get_text() not in '시간']

                # 1시간이 넘어간 뉴스는 긁어오지 않음 (발견되면 크롤링 종료)
                if len(upload) != 5:
                    flag = False
                    news = news[:len(upload)]
                    urls = urls[:len(upload)]
                    press = press[:len(upload)]

                temp_df = pd.DataFrame({"News":news, "Url":urls, "Press":press, "Time":upload})
                News_DataFrame = pd.concat([News_DataFrame, temp_df], axis=0, ignore_index=True)
                News_DataFrame.dropna(axis=0, inplace=True)

        # 섹션 별 크롤링 시간 측정 end
        end = time.time()
        total_crawling_time = total_crawling_time + (end - start)        
        
        # 데이터 프레임 csv 파일로 저장
        title = section + ".csv"
        News_DataFrame.to_csv(title, sep=",", encoding='utf-8-sig', index=False)
        print("Crawling " + date + " " + str(now.hour) + " " + section + " News :", end-start)
        
    print(date + " " + str(now.hour) + " total time :", total_crawling_time)
    print(date + " " + str(now.hour) + " average Time :", total_crawling_time / len(Section_ID))
    print("────────────────────────────")

In [ ]:
from datetime import datetime, timedelta
import pause, sys

now = datetime.now()
print("Automation Crawling Start : " + str(now.date()) + " " + str(now.hour) + ":" + str(now.minute) + ":" + str(now.second))

while True:
    d = datetime.now()
    t = d + timedelta(hours=1)
    pause.until(datetime(t.year, t.month, t.day, t.hour, 0, 0))
    crawling()

Automation Crawling Start : 2021-02-09 12:23:10
Start Crawling : 2021-02-09 13
Crawling 20210209 13 Political News : 0.8855631351470947
Crawling 20210209 13 Economic News : 0.8524665832519531
Crawling 20210209 13 Society News : 0.8570346832275391
Crawling 20210209 13 Culture News : 0.861732006072998
Crawling 20210209 13 International News : 0.9115674495697021
Crawling 20210209 13 Science News : 0.8757727146148682
20210209 13 total time : 5.244136571884155
20210209 13 average Time : 0.8740227619806925
────────────────────────────
Start Crawling : 2021-02-09 14
Crawling 20210209 14 Political News : 0.9305093288421631
Crawling 20210209 14 Economic News : 0.8298301696777344
Crawling 20210209 14 Society News : 0.8028016090393066
Crawling 20210209 14 Culture News : 0.8267898559570312
Crawling 20210209 14 International News : 0.8068444728851318
Crawling 20210209 14 Science News : 0.8337705135345459
20210209 14 total time : 5.030545949935913
20210209 14 average Time : 0.8384243249893188
──────